# stable-diffusion-2

* https://huggingface.co/stabilityai/stable-diffusion-2
* https://proximacentaurib.notion.site/e28a4f8d97724f14a784a538b8589e7d?v=ab624266c6a44413b42a6c57a41d828c

In [1]:
import os
import random
import pandas as pd

In [2]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

model_id = "stabilityai/stable-diffusion-2"

# Use the Euler scheduler here instead
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, safety_checker=None)
pipe = pipe.to("cuda")

/storage/local/stablediff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 13 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 125925.99it/s]
/storage/local/stablediff/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [3]:
def prompt(prompt, n=1, style=None, path='.'):    
    if style != None:
        prompt += ' by %s'%style    
    for c in range(n):
        print (prompt)
        image = pipe(prompt, negative_prompt='disfigured').images[0]
        if not os.path.exists(path):
            os.makedirs(path)
        i=1
        imgfile = os.path.join(path,prompt[:60]+'_%s.png' %i)
        while os.path.exists(imgfile):
            i+=1
            imgfile = os.path.join(path,prompt[:90]+'_%s.png' %i)
        image.save(imgfile,'png')        
    return image

In [4]:
def get_words():
    import requests
    word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
    response = requests.get(word_site)
    w = response.content.splitlines()
    w = [i.decode() for i in w]
    return w

randwords = get_words()

In [5]:
def random_image(artist,n=1,k=3):
    for i in range(n):
        words = random.choices(randwords,k=k)
        print (words)
        txt = ' '.join(words)
        prompt(txt, style=artist, path=artist)

In [ ]:
prompt("a beautiful oil painting of a bowl of fruit at evening time on a table",n=5)

In [ ]:
prompt("liffey in Dublin",style='willem kalf')

In [ ]:
prompt("american eagle flying over europe with USA flag, painting, dramatic")

In [ ]:
prompt("man sitting in sunlit pasture on chair",style="baes firmin")

In [ ]:
prompt("river mountains distant cloudy watercolor pencil",style='tolkien',path='tolkien')

In [ ]:
prompt('giger alien derelict ship', n=5, style='moebius',path='moebius')

In [ ]:
style=''
prompt('3d cartoon render of cow using computer, trending on artstation,',n=15)

## look at varying styles

In [ ]:
styles = pd.read_csv('stable_diffusion_artist_styles.csv')
names = styles.sample(30).Name
p='julian assange'
for name in names:
    prompt(p+'',style=name,path=p)

In [ ]:
pres=['ronald reagan','barack obama','john f kennedy','bill clinton','donald trump','george w bush','richard nixon','jimmy carter']
artist='grant wood'
for p in pres:
    prompt(p,style=artist,path='presidents',n=1)

## random combination of a dictionary of words

In [ ]:
#artist='ernst haeckel'
artist='paul hedley'
#artist='robert bateman'
#rtist='albert benois'
#artist='antoine blanchard'
#artist='vincent van gogh'
#artist='phillipe druillet'
#artist='ralph mcquarrie'
#artist='moebius'
#artist='hippolyte flandrin'
#artist='gifford beale'
#artist='greg rutkowski'
#artist='vladimir volegov'
#artist='albrecht anker'
artist='norman rockwell'

themes = {'tolkien':(['gondor','isengard','lothlorien','mordor','rohan','numenor'],['mountain','forest','tower','ruins'],['dwarves','orc','hobbit','gandalf','elf','eagle','bear']),
        'ireland':(['ireland','dublin'],['monastic','castle','abbey','celtic','shamrock','green','ruins','cross'],['tree','oak','birch','moon','mountain','cliffs']),
        'star trek':(['star trek','enterprise'],['scifi','70s','retro','vulcan']),
        'nature':(['pond','forest','wood'],['landscape','mountain','valley'],['summer','spring','autumn']),
        'imperialism':(['USA','nato','europe'],['hates','loves'],['war','nazi','peace','china','russia']),
        'covid':(['surgical mask','vaccine'],['covid','virus','bacteria'],['obedience','locked up','lab','disease']),          
        'wotw':(['hg wells','war of the worlds'],['invasion','martian','scifi'])}
t='nature'
for i in range(10):
    words = themes[t]
    #w = set(random.choices(words,k=4))
    w = [random.choice(i) for i in words]
    txt = '%s_'%t +' '.join(w)
    prompt(txt, style=artist, path=artist)

In [ ]:
#random_image('ernst haeckel',10)
random_image('inessa garmash',10)

## film scenes

In [ ]:
artist='robert bateman'
modf = pd.read_csv('movies.csv')
for p in modf.sample(10).name:
    prompt(p, path='movie scenes',style=artist)

## panel app

In [ ]:
import panel as pn
import panel.widgets as pnw